In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
llm= ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x74c7b7b89c10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x74c7b7b8a780>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions known for their loyalty and friendliness",
        metadata = {"source":"mammal-pets-doc"}
    ),
      Document(
        page_content="Cats are independent animals but can form strong bonds with their owners",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birds that can mimic human speech and enjoy social interaction",
        metadata={"source": "bird-pets-doc"}
    ),
    Document(
        page_content="Turtles are low-maintenance pets but require a specific habitat and diet for longevity",
        metadata={"source": "reptile-pets-doc"}
    )
]

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-V2")

/home/ayush/Code/ML-Projects/Gemini/Langchain/env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/ayush/Code/ML-Projects/Gemini/Langchain/env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents,embedding=embeddings)
vectorstore

In [7]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals but can form strong bonds with their owners'),
  0.9499919414520264),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness'),
  1.5372059345245361),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds that can mimic human speech and enjoy social interaction'),
  1.643890142440796),
 (Document(metadata={'source': 'reptile-pets-doc'}, page_content='Turtles are low-maintenance pets but require a specific habitat and diet for longevity'),
  1.6765367984771729)]

In [8]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['cat',"dog"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals but can form strong bonds with their owners')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness')]]

In [9]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(['cat',"dog"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent animals but can form strong bonds with their owners')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions known for their loyalty and friendliness')]]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only 
{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human",message)])


rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response = rag_chain.invoke("tell me about dogs")
print(response.content)